In [2]:
import pandas as pd
import numpy as np
import re

import unicodedata
from nltk.corpus import stopwords






## Creamos funciones para preparar datasets

#### Champú

In [5]:
def Preprocessing_champu():
    df1 = pd.read_csv('champú sólido hombre.csv')
    df2 = pd.read_csv('champú sólido hombre details.csv')

    df1 = df1.drop_duplicates(subset=['ASIN'], keep='first')
    df2 = df2.drop_duplicates(subset=['ASIN'], keep='first')

    df_champu = pd.merge(df1, df2, on='ASIN', how='left')

    df_champu = df_champu.drop(columns="Seller")
    
    return df_champu

In [6]:
df_champu = Preprocessing_champu()

In [10]:
df_champu.shape

(186, 17)

#### Jabón

In [14]:
def Preprocessing_Jabon():
    df1 = pd.read_csv('jabon solido hombre.csv')
    df2 = pd.read_csv('jabon solido hombre details.csv')

    df1 = df1.drop_duplicates(subset=['ASIN'], keep='first')
    df2 = df2.drop_duplicates(subset=['ASIN'], keep='first')

    df_jabon = pd.merge(df1, df2, on='ASIN', how='left')
    df_jabon = df_jabon.drop(columns="Product_Information")
    df_jabon = df_jabon.drop(columns="Unnamed: 11")
    
    return df_jabon

In [16]:
df_jabon = Preprocessing_Jabon()

In [18]:
df_jabon.shape

(196, 17)

#### Exfoliante

In [24]:
def Preprocessing_Exfoliante():
    df1 = pd.read_csv('exfoliante solido hombre.csv')
    df2 = pd.read_csv('exfoliante solido hombre details.csv')

    df1 = df1.drop_duplicates(subset=['ASIN'], keep='first')
    df2 = df2.drop_duplicates(subset=['ASIN'], keep='first')

    df_exfoliante = pd.merge(df1, df2, on='ASIN', how='left')
   
    df_exfoliante = df_exfoliante.drop(columns=['Current_Time_y', 'Image_URL_1', 'Image_URL_2', 'Image_URL_3','Image_URL_4', 'Image_URL_5', 'Image_URL_6'])
    df_exfoliante = df_exfoliante.drop(columns=['Item_Weight','Product_Details', 'Item_Model_Number','Product_Information','Precio_Envío', 'Style','Color','Seller','Number_of_Answered_Questions','Web_Page_URL'])
    return df_exfoliante

In [26]:
df_exfoliante = Preprocessing_Exfoliante()

In [28]:
df_exfoliante.shape

(172, 17)

#### Valores Repetidos

In [31]:

def eliminar_duplicados_productos(df):
    
    # Verificar si la columna existe
    if 'Product_Description' not in df.columns:
        raise ValueError("El DataFrame no contiene la columna 'Product_Description'")

    copy = df.copy()
    
    # Calcular duplicados antes
    duplicados = copy.duplicated(subset=['Product_Description'], keep='first').sum()
    
    # Eliminar duplicados (keep='first' conserva la primera ocurrencia)
    copy = copy.drop_duplicates(subset=['Product_Description'], keep='first')
    

    df = copy 
    
    return df

#### Null Data Processing

In [34]:
# def clean_text(text):
#     if not isinstance(text, str):  # Por si hay NaN o números
#         return ""
#     # Elimina URLs
#     text = re.sub(r'https?://\S+|www\.\S+', '', text)
#     # Eliminar tildes y caracteres especiales
#     text = unicodedata.normalize("NFD", text)
#     text = text.encode("ascii", "ignore").decode("utf-8")
#     # Elimina menciones (@) y hashtags (#)
#     text = re.sub(r'@\w+|#\w+', '', text)
#     # Elimina caracteres especiales (excepto letras, números y espacios)
#     text = re.sub(r'[^\w\s]', '', text)
#     # Convierte a minúsculas
#     text = text.lower()
#     # Elimina espacios extras
#     text = ' '.join(text.split())
#     return text

In [38]:
def Preprocessing_Nulls(df):

    copy = df.copy()
    # Crear dataset con nulos (usando .loc para evitar warnings)
    Nulos = copy.loc[copy["Star_Rating_y"].isna()]  # Copia explícita
    Nulos.to_csv("datos_nulos.csv", index=False, encoding="utf-8")

    # Eliminar filas con nulos (esto crea un nuevo DataFrame)
    copy = copy.dropna(subset=["Star_Rating_y", "Price_x"])

    # Concatenar columnas con .loc
    copy.loc[:, 'Product_Description'] = (
        copy['Title_x'].fillna('') + ' ' +
        copy['Bullet_Points'].fillna('') + ' ' +
        copy['Product_Description'].fillna('')
    )


    
    # # Procesar texto
    # copy.loc[:, 'Product_Description'] = copy['Product_Description'].apply(clean_text)

    copy["Inventory"] = copy["Inventory"].apply( lambda x:0 if x == "No disponible." else 1)  # Cambiamos de tipo a numérica , los valores NaN se corresponden con elementos en stock, lo ponemos a 1
    

    # Eliminar columnas (usando drop con axis=1 para columnas)
    copy = copy.drop(columns=["Title_x", "Bullet_Points"], axis=1)  

    df= copy

    return df
    

#### Cambios de tipo y modificación de valores

In [41]:
def Transform_data(df):
    
    copy = df.copy()

    copy = copy.rename(columns={
    'Inventory': 'Stock',  
    'Price_x': 'Price',
    'Star_Rating_y': 'Star_Rating',
    'Number_of_Reviews_y': 'Reviews',
    'Clasificacion_en_los_mas_vendidos_de_Amazon': 'Ranking_Amazon'
    })

    copy["Stock"] = copy["Stock"].apply( lambda x:0 if x == "No disponible." else 1)
    
    copy['Price'] = copy['Price'].str.replace('€', '').str.replace(',', '.').astype(float)  #Cambio de tipo string a float y quitamos el simbolo de divisa

    copy['Reviews'] = copy['Reviews'].str.extract(r'(\d+)').astype(int)  

    copy['Star_Rating'] = (
    copy['Star_Rating']
    .str.extract(r'(\d+[,.]?\d*)')[0]  # Captura "4,4", "5", "5.0", etc.
    .str.replace(',', '.')              # Reemplaza coma por punto
    .astype(float)                      # Convierte a float
    )


    df= copy

    return df
    


### Función Preprocessing

In [46]:
def Preprocessing():
    
    df_champu = Preprocessing_champu()           #Ejecutamos las funciones que crean los datasets de productos por tipo
    df_jabon = Preprocessing_Jabon()
    df_exfoliante = Preprocessing_Exfoliante()

    df_champu["Tipo"] = "champu"                #Creamos la culumna tipo para almacenar el tipo de producto proveniente de cada dataset
    df_jabon["Tipo"] = "jabon"
    df_exfoliante["Tipo"] = "exfoliante"

    df = pd.concat([df_champu, df_jabon, df_exfoliante], axis=0, ignore_index=True)    #Unimos los datasets en uno sólo

    df = df.drop(columns=["Keyword","Brand","Detail_Page_URL","Number_of_Reviews_x","Price_y"])   #Borramos las columnas que no aporten datos relevantes o esten repetidas
    df = df.drop(columns=["ASIN","Current_Time","Star_Rating_x","Title_y","Current_Time_x"])

    df = (df.pipe(Preprocessing_Nulls).pipe(Transform_data).pipe(eliminar_duplicados_productos))

    
    
    df_datosaux = df[["Stock","Ranking_Amazon"]]
    df = df.drop(columns = ["Stock","Ranking_Amazon"])
    
    df.to_csv("datos_productos.csv", index=False, encoding="utf-8")

    return df,df_datosaux

In [49]:
df,df_datosaux = Preprocessing()

,Price,Star_Rating,Reviews,Product_Description,Tipo
0,6.95,4.4,477,"L'Oréal Men Expert Champú sólido para hombres,...",champu
2,11.99,4.2,455,Störtebekker® Premium Champú sólido Sándalo (1...,champu
3,14.99,5.0,12,Champú Sólido sin Parabenos ni Sulfatos-Vegano...,champu
4,11.99,4.4,64,RAW REVIVAL - Champú solido Romero | Anticaspa...,champu
5,11.99,4.4,24,Störtebekker® Premium Champú sólido Tortuga - ...,champu
...,...,...,...,...,...
540,21.50,5.0,2,Exfoliante Exfoliante de Bambú de Phyt's Men 1...,exfoliante
542,42.00,4.2,131,Paula’s Choice RESIST Antiedad 2% BHA Exfolian...,exfoliante
543,20.50,4.1,194,Himalaya Herbals Exfoliante de albaricoque 50 ...,exfoliante
545,70.52,4.3,649,Vasanti – Rejuvenecedor facial enzimático Brig...,exfoliante


In [54]:
df.isna().sum()

Price                  0
Star_Rating            0
Reviews                0
Product_Description    0
Tipo                   0
dtype: int64

In [58]:
df.shape

(395, 5)

### Tratamiento de NaN

In [182]:
df.isna().sum()

Price                  0
Star_Rating            0
Reviews                0
Product_Description    0
Tipo                   0
dtype: int64

In [184]:
porcentaje_nan = (df.isna().mean() * 100).round(2)

print("\nPorcentaje de NaN por columna (%):")
print(porcentaje_nan)


Porcentaje de NaN por columna (%):
Price                  0.0
Star_Rating            0.0
Reviews                0.0
Product_Description    0.0
Tipo                   0.0
dtype: float64


In [186]:
df


,Price,Star_Rating,Reviews,Product_Description,Tipo
0,6.95,4.4,477,loreal men expert champu solido para hombres j...,champu
2,11.99,4.2,455,stortebekker premium champu solido sandalo 1 p...,champu
3,14.99,5.0,12,champu solido sin parabenos ni sulfatosvegano ...,champu
4,11.99,4.4,64,raw revival champu solido romero anticaspa der...,champu
5,11.99,4.4,24,stortebekker premium champu solido tortuga cha...,champu
...,...,...,...,...,...
540,21.50,5.0,2,exfoliante exfoliante de bambu de phyts men 10...,exfoliante
542,42.00,4.2,131,paulas choice resist antiedad 2 bha exfoliante...,exfoliante
543,20.50,4.1,194,himalaya herbals exfoliante de albaricoque 50 ...,exfoliante
545,70.52,4.3,649,vasanti rejuvenecedor facial enzimatico bright...,exfoliante


In [189]:
df_datosaux

,Stock,Ranking_Amazon
0,1,nº62.734 en Belleza ( Ver el Top 100 en Bellez...
2,1,nº64.795 en Belleza ( Ver el Top 100 en Bellez...
3,1,nº11.704 en Belleza ( Ver el Top 100 en Bellez...
4,1,nº4.764 en Belleza ( Ver el Top 100 en Belleza...
5,1,nº187.189 en Belleza ( Ver el Top 100 en Belle...
...,...,...
540,1,NaN
542,1,nº142.183 en Belleza ( Ver el Top 100 en Belle...
543,1,NaN
545,1,nº241.472 en Belleza ( Ver el Top 100 en Belle...


In [192]:
df

,Price,Star_Rating,Reviews,Product_Description,Tipo
0,6.95,4.4,477,loreal men expert champu solido para hombres j...,champu
2,11.99,4.2,455,stortebekker premium champu solido sandalo 1 p...,champu
3,14.99,5.0,12,champu solido sin parabenos ni sulfatosvegano ...,champu
4,11.99,4.4,64,raw revival champu solido romero anticaspa der...,champu
5,11.99,4.4,24,stortebekker premium champu solido tortuga cha...,champu
...,...,...,...,...,...
540,21.50,5.0,2,exfoliante exfoliante de bambu de phyts men 10...,exfoliante
542,42.00,4.2,131,paulas choice resist antiedad 2 bha exfoliante...,exfoliante
543,20.50,4.1,194,himalaya herbals exfoliante de albaricoque 50 ...,exfoliante
545,70.52,4.3,649,vasanti rejuvenecedor facial enzimatico bright...,exfoliante


In [195]:
duplicados = df[df.duplicated(subset=['Product_Description'], keep=False)]
print(duplicados)

Empty DataFrame
Columns: [Price, Star_Rating, Reviews, Product_Description, Tipo]
Index: []
